# Training the model

In [ ]:
import tensorflow as tf
import collections
#from tensorflow import keras
from keras import layers
from keras.layers import Input,Dense,Activation,ZeroPadding2D,BatchNormalization,Flatten,Conv2D
from keras.layers import AveragePooling2D,MaxPooling2D,Dropout,GlobalMaxPool2D,GlobalAveragePooling2D
from keras.models import Model
from keras.preprocessing import image
from keras.utils import layer_utils
from keras.utils.data_utils import get_file
from keras.applications.imagenet_utils import preprocess_input
import pydot
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
from keras.utils import plot_model

import keras.backend as K
K.set_image_data_format('channels_last')
#inputshape=(height,width,deep)

import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow

import h5py
import os 
from PIL import Image
import sklearn
from sklearn import preprocessing
import scipy
import numpy as np

In [ ]:
train_dataset = h5py.File("./datasets_soilandfire_20200614.h5",'r')
X_train = np.array(train_dataset['x_train'][:])
Y_train = np.array(train_dataset['y_train'][:])
X_test = np.array(train_dataset['x_test'][:])
Y_test = np.array(train_dataset['y_test'][:])
train_dataset.close()
#train_data= create_dataset(X_train,Y_train)

In [ ]:
class_names=["fire","soil"]
print(len(X_train))
for i in range(25):
    print(np.squeeze(X_train[i]),np.sum(X_train[i]==1))
    print(class_names[int(Y_train[i])])

In [ ]:
print("number of traning examples = "+str(X_train.shape[0]))
print("number of test examples = "+str(X_test.shape[0]))
print("X_train shape:" + str(X_train.shape))
print("Y_train shape:" + str(Y_train.shape))
print("X_test shape:" + str(X_test.shape))
print("Y_test shape:" + str(Y_test.shape))
train_data=(X_train,Y_train)
test_data=(X_test,Y_test)

In [ ]:
#Build a model
def Pimodel(input_shape):
    X_input = Input(shape=input_shape)
    X=ZeroPadding2D(padding=(1,1))(X_input)
    X=Conv2D(1,kernel_size=(6,6),strides=(6,6),use_bias=None)(X)
    X=Activation('relu')(X)
    X=MaxPooling2D(pool_size=(2,1))(X)
    #X=Conv2D(1,kernel_size=(1,2),strides=(1,1),use_bias=None)(X)
    X=Flatten()(X)
    Y=Activation('softmax')(X)
    model=Model(inputs=X_input,outputs=Y,name="JSmodel")
    return model

In [ ]:
picmodel = Pimodel((10,10,1))

In [ ]:
import keras
picmodel.compile(optimizer=keras.optimizers.Adam(lr=0.001,beta_1=0.9,beta_2=0.999,epsilon=1e-08,decay=0.0),loss='sparse_categorical_crossentropy',metrics=['accuracy'])

In [ ]:
picmodel.summary()

In [ ]:
history=picmodel.fit(x=X_train,y=Y_train,batch_size=2000,epochs=300,validation_data=(X_test,Y_test))

In [ ]:
preds = picmodel.evaluate(x=X_test,y=Y_test)

print('val Loss=' + str(preds[0]))
print(picmodel.name+"Accuracy="+str(preds[1]))

In [ ]:
for i in range(len(history.history["accuracy"])):
    print(history.history["accuracy"][i],history.history["val_accuracy"][i])

In [ ]:
plt.plot(history.history["accuracy"])
plt.plot(history.history['val_accuracy'])
plt.title('Two-categrorical model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.ylim((0, 1))
plt.legend(['train accuracy', 'val accuracy'], loc='center right')
plt.savefig("Two-categrorical_acc.png")

In [ ]:
for i in range(len(history.history["loss"])):
    print(history.history["loss"][i],history.history["val_loss"][i])

In [ ]:
plt.plot(history.history["loss"])
plt.plot(history.history['val_loss'])
plt.title('Two-categrorical model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train loss', 'val loss'], loc='center right')
plt.savefig("Two-categrorical_loss.png")

# output the model weight and biases

In [ ]:
kernel_weights=picmodel.layers[2].get_weights()
for i in np.squeeze(kernel_weights):
    print(i)

In [ ]:
from keras import backend as K
from keras.models import load_model
import numpy as np
def get_layer_output(model,x,index=-1):
    layer = K.function([model.input],[model.layers[index].output])
    return layer([x])[0]
input_y=np.load("./datasets/fire/120DP2ooodataset_original_fire(7).png_74c3dbc8-bacc-4d3e-a265-598ba6179cd4.png.npy")
input_x = np.expand_dims(input_y,axis=0)
input_x = np.expand_dims(input_x,axis=3)
layer_0 = get_layer_output(picmodel,input_x,index=0)
layer_1 = get_layer_output(picmodel,input_x,index=1)
layer_2 = get_layer_output(picmodel,input_x,index=2)
layer_3 = get_layer_output(picmodel,input_x,index=3)
layer_4 = get_layer_output(picmodel,input_x,index=4)
layer_5 = get_layer_output(picmodel,input_x,index=5)
print(np.squeeze(layer_0)
print(np.squeeze(layer_1))
print(np.squeeze(layer_2))
print(np.squeeze(layer_3))
print(np.squeeze(layer_4))
print(np.squeeze(layer_5))

In [ ]:
print(picmodel.summary())
for i in np.squeeze(kernel_weights):
    print(i)

kernel_weights=np.squeeze(picmodel.get_weights())
a1=np.squeeze(kernel_weights)[0:12,0:6].flatten()
print(a1)
N=abs(a1.flatten())[np.argmax(abs(a1.flatten()))]
print(N)
a1_nor=a1.flatten()/N
noraml_kernel_weights=(kernel_weights/N)
for j in noraml_kernel_weights:
    print(j)

In [ ]:
from keras import backend as K
from keras.models import load_model
import numpy
import os,sys
def get_layer_output(model,x,index=0):
    layer = K.function([model.input],[model.layers[index].output])
    return layer([x])[0]

path="./datasets/fire/"
imlist=os.listdir(path)
fire_t=[]
Fire_t=[]
fn_t=[]
for elef in imlist:
    input_x = np.array(np.load(r"./datasets/fire/"+elef))
    input_x = np.expand_dims(input_x,axis=0)
    input_x = np.expand_dims(input_x,axis=3)
    layer_1 = get_layer_output(picmodel,input_x,index=1)
    layer_2 = get_layer_output(picmodel,input_x,index=2)
    layer_3 = get_layer_output(picmodel,input_x,index=3)
    layer_4 = get_layer_output(picmodel,input_x,index=4)
    layer_5 = get_layer_output(picmodel,input_x,index=5)
    fire_t.append(class_names[np.argmax(layer_4)])
    if class_names[np.argmax(layer_4)]=="fire":
        FN_t=[]
        FN_t.append(elef)
        FN_t.append(layer_2.flatten())
        FN_t.append(layer_5.flatten())
        fn_t.append(FN_t)

f1=collections.Counter(fire_t)
print(len(fire_t),collections.Counter(fire_t))

In [ ]:
def takesecond(elem):
    return elem[1][0]
fn_t.sort(key=takesecond,reverse=True)
print(len(fn_t))
for i in fn_t:
    print(i[0])
    print(i[1]/N)

In [ ]:
path="./datasets/soil/"
imlist=os.listdir(path)
soil_T=[]
Soil_T=[]
sn_t=[]
for elef in imlist:
    input_x = np.array(np.load(r"./datasets/soil/"+elef))
    input_x = np.expand_dims(input_x,axis=0)
    input_x = np.expand_dims(input_x,axis=3)
    layer_1 = get_layer_output(picmodel,input_x,index=1)
    layer_2 = get_layer_output(picmodel,input_x,index=2)
    layer_3 = get_layer_output(picmodel,input_x,index=3)
    layer_4 = get_layer_output(picmodel,input_x,index=4)
    layer_5 = get_layer_output(picmodel,input_x,index=5)
    #print(class_names[np.argmax(layer_6)])
    soil_T.append(class_names[np.argmax(layer_4)])
    if class_names[np.argmax(layer_4)]=="soil":
        SN_t=[]
        SN_t.append(elef)
        SN_t.append(layer_2.flatten())
        SN_t.append(layer_4.flatten())
        sn_t.append(SN_t)
        Soil_T.append(elef)
print(len(soil_T))
s1=collections.Counter(soil_T)
print(collections.Counter(soil_T))

In [ ]:
def takesecond(elem):
    return elem[0][1]
sn_t.sort(key=takesecond,reverse=True)
print(len(sn_t))
for i in sn_t:
    print(i[0])
    print(i[1]/N)

In [ ]:
import seaborn as sns
import pandas as pd
sns.set(style="darkgrid")
f2=dict(f1)
print(f2)
s2=dict(s1)
print(s2)
dict1=pd.Series(f1).reindex(["fire","soil"])
print(dict1.index)
print(dict1.values)
#print(sns.barplot(dict1.index,dict1.values))
#print(dict1.plot.bar())
dict2=pd.Series(s2).reindex(["fire","soil"])
print(sns.barplot(dict1.index,dict1.values))
plt.title("fire recongnition")
plt.savefig("fire_rec.png")